In [13]:
from read_data import get_train_data
from model import unet_model
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

In [2]:
# get train_data
train_img, train_mask = get_train_data()

Getting and resizing train images and masks ... 
669/670 [============================>.] - ETA: 0s

In [11]:
# Splitting into train and validation
X_train, X_valid, y_train, y_valid = train_test_split(train_img, train_mask, test_size=0.15, random_state=2020)

In [3]:
# get u_net model using elu activation func
unet_elu = unet_model(activation="elu")

In [17]:
unet_elu.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 256, 256, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 16) 448         lambda_1[0][0]                   
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 256, 256, 16) 0           conv2d_1[0][0]                   
____________________________________________________________________________________________

In [14]:
# Specify EarlyStop, Learning Rate decreasing and Model Checkpoint
callbacks = [
    EarlyStopping(monitor='dice_coef', mode='max', patience=4, verbose=1),
    ReduceLROnPlateau(factor=0.1, patience=3, min_lr=0.00001, verbose=1),
    ModelCheckpoint('model_weights.h5', verbose=1, save_best_only=True, save_weights_only=True)
]

In [16]:
# Training
unet_elu.fit(X_train, y_train, batch_size=16, epochs=50 
                  validation_data=(X_valid, y_valid), callbacks=callbacks)

Train on 569 samples, validate on 101 samples
Epoch 1/3
569/569 [==============================] - 293s 516ms/step - loss: 0.3878 - dice_coef: 0.3125 - val_loss: 0.1935 - val_dice_coef: 0.5521

Epoch 00001: val_loss improved from inf to 0.19346, saving model to model_weights.h5
Epoch 2/3
569/569 [==============================] - 307s 540ms/step - loss: 0.1966 - dice_coef: 0.6421 - val_loss: 0.1368 - val_dice_coef: 0.7462

Epoch 00002: val_loss improved from 0.19346 to 0.13682, saving model to model_weights.h5
Epoch 3/3
569/569 [==============================] - 329s 579ms/step - loss: 0.1511 - dice_coef: 0.7279 - val_loss: 0.0937 - val_dice_coef: 0.8065

Epoch 00003: val_loss improved from 0.13682 to 0.09373, saving model to model_weights.h5
